# Olympic Games. Data exploration

#### 0. Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

## 1. Load Data

In [8]:
athletes = pd.read_csv("../dataset/athletes.csv").drop_duplicates()
athletes.head(1)

,id,name,gender,age,height,weight
0,51,Nstor Abad Sanjun,M,23,167.0,64.0


In [ ]:
country = pd.read_csv("../dataset/countries.csv").drop_duplicates()
country.head(1)

In [ ]:
stats = pd.read_csv("../dataset/country_stats.csv").drop_duplicates()
stats.head(1)

In [ ]:
summer = pd.read_csv("../dataset/summer_games.csv").drop_duplicates()
summer.head(1)

In [ ]:
winter = pd.read_csv("../dataset/winter_games.csv").drop_duplicates()
winter.head(1)

## 2. Pre-process Data

### 2.1 Basic transformations

In [ ]:
athletes = athletes.rename(columns={"id":"athlete_id"})

Drop the youngest entry for athlete 127594 because it is duplicated.

In [ ]:
athletes = athletes.loc[~((athletes["athlete_id"] == 127594) & (athletes["age"] == 28)), :]

Drop NA values and separate country code from country name.

In [ ]:
country = country.dropna()
country[['country_code','country_name']] = country["country"].str.split("-",expand=True,)
country = country.drop(["country"], axis=1)
country = country.rename(columns={"id":"country_id"})

Replace missing values and transform the year to YYYY format.

In [ ]:
stats = stats.replace(np.NaN, 0)
stats["year"] = stats["year"].astype(str).str[0:4].astype(int)

Transform the year to YYYY formats and replace NaNs in `medal` with "No Medal".

In [ ]:
summer["year"] = summer["year"].astype(str).str[0:4].astype(int)
summer["medal"] = summer["medal"].replace(np.NaN, "No medal")

Transform the year to YYYY formats and replace NaNs in `medal` with "No Medal".

In [ ]:
winter["year"] = winter["year"].astype(str).str[0:4].astype(int)
winter["medal"] = winter["medal"].replace(np.NaN, "No medal")
winter.head(1)

### 2.2 Joins

First, join the `country` dataset with the `stats` dataset by `country_id`:

In [ ]:
country_stats = country.merge(stats, on="country_id", how="left")
country_stats.head(1)

Then, add the country information to the `athletes`. 

The relation `country_id` to `athlete_id` is contained in the `summer` and `winter` datasets, so first we extract it and get the 4215 athletes with their countries.

In [ ]:
ath_co_summer = summer[["athlete_id", "country_id"]].drop_duplicates()
ath_co_winter = winter[["athlete_id", "country_id"]].drop_duplicates()
ath_co = pd.concat([ath_co_summer, ath_co_winter]).drop_duplicates()

Add the country ID to the `athletes`:

In [ ]:
athletes = athletes.merge(ath_co, how="left")
athletes.head(1)

Join with the country data:

In [ ]:
athletes_country = athletes.merge(country_stats, how="left")
athletes_country.head(1)

Finally we can add the athletes´ information to the `summer` and `winter` datasets, making sure that we only keep the year of the games.

In [ ]:
summer_full = summer.merge(athletes_country, how="left")
summer_full.head(1)

In [ ]:
winter_full = winter.merge(athletes_country, how="left")
winter_full.head(1)

## 3. Exercises

### 3.2 

Number of athletes of 3 top sports in summer games:

In [ ]:
summer_full[["sport", "athlete_id"]].drop_duplicates().groupby("sport").size()

Or by event:

In [ ]:
summer_full[["event", "athlete_id"]].drop_duplicates().groupby("event").size().head(3)

### 3.3

Create a report that shows every sport's number of unique events and unique athletes

**Summer**

In [ ]:
sport_event_summer = summer_full[["sport", "event"]].drop_duplicates().groupby(["sport"]).size()
sport_event_summer = sport_event_summer.reset_index()
sport_event_summer = sport_event_summer.rename(columns={0:"nr_unique_events"})
sport_event_summer

In [ ]:
sport_ath_summer = summer_full[["sport", "athlete_id"]].drop_duplicates().groupby(["sport"]).size()
sport_ath_summer = sport_ath_summer.reset_index()
sport_ath_summer = sport_ath_summer.rename(columns={0:"nr_athletes"})
sport_ath_summer

In [ ]:
sport_events_ath = sport_event_summer.merge(sport_ath_summer)
sport_events_ath

**Winter**

In [ ]:
sport_event_winter = winter_full[["sport", "event"]].drop_duplicates().groupby(["sport"]).size()
sport_event_winter = sport_event_winter.reset_index()
sport_event_winter = sport_event_winter.rename(columns={0:"nr_unique_events"})
sport_event_winter

In [ ]:
sport_ath_winter = winter_full[["sport", "athlete_id"]].drop_duplicates().groupby(["sport"]).size()
sport_ath_winter = sport_ath_winter.reset_index()
sport_ath_winter = sport_ath_winter.rename(columns={0:"nr_athletes"})
sport_ath_winter

In [ ]:
sport_events_ath = sport_event_winter.merge(sport_ath_winter)
sport_events_ath

### 3.4

Age of oldest athlete by region.

In [ ]:
athletes_country[["region", "age"]].groupby("region").max("age").reset_index()

### 3.5

Create a report that shows the unique number of events held for each sport

In [ ]:
sport_event_summer = summer_full[["sport", "event"]].drop_duplicates().groupby(["sport"]).size()
sport_event_summer = sport_event_summer.reset_index()
sport_event_summer = sport_event_summer.rename(columns={0:"nr_unique_events"})
sport_event_summer["season"] = "summer"
sport_event_summer

In [ ]:
sport_event_winter = winter_full[["sport", "event"]].drop_duplicates().groupby(["sport"]).size()
sport_event_winter = sport_event_winter.reset_index()
sport_event_winter = sport_event_winter.rename(columns={0:"nr_unique_events"})
sport_event_winter["season"] = "winter"
sport_event_winter

In [ ]:
sport_event = pd.concat([sport_event_summer, sport_event_winter])
sport_event = sport_event.sort_values(by="nr_unique_events", ascending=False)
sport_event

### 3.6

Total bronze medals from summer games.

In [ ]:
summer[["medal"]].value_counts()

Bronze medals by country:

In [ ]:
summer_full.loc[summer_full["medal"] == "Bronze", ["country_name", "medal"]].value_counts()

### 7. Interesting insights

***
#### Summary

* The USA was a major contributor of athletes to the olympic games (over 5% of the participants), both in summer and in winter.  

* The USA are clearly superior in summer. They won 72 medals every 100 athletes. 

* Norway is clearly superior in winter. They won 73 medals every 100 athletes.

* I suspect that countries in high latitudes are better at winter games.
***

I will use the `summer_full` and `winter_full` datasets (summer and winter games information, enriched with the country stats and athlete information).

First I create a dataframe with the amount of medals by country for both winter and summer games. 

In [ ]:
medal_country_s = summer_full[["country_name", "medal"]].groupby(["country_name", "medal"]).size()
medal_country_s = pd.DataFrame(medal_country_s).reset_index().rename(columns={0:"nr_medals"})
medal_country_s = medal_country_s.pivot(index="country_name", columns="medal", values="nr_medals")
medal_country_s = medal_country_s.reset_index().replace(np.NaN, 0)

In [ ]:
medal_country_w = winter_full[["country_name", "medal"]].groupby(["country_name", "medal"]).size()
medal_country_w = pd.DataFrame(medal_country_w).reset_index().rename(columns={0:"nr_medals"})
medal_country_w = medal_country_w.pivot(index="country_name", columns="medal", values="nr_medals")
medal_country_w = medal_country_w.reset_index().replace(np.NaN, 0)

Then calculate the number of athletes by country:

In [ ]:
nr_ath = summer_full[["country_name", "athlete_id"]].drop_duplicates().groupby("country_name").size()
nr_ath = pd.DataFrame(nr_ath.reset_index()).rename(columns={0:"nr_participants"})
medal_country_s = medal_country_s.merge(nr_ath, how="left")

In [ ]:
nr_ath = winter_full[["country_name", "athlete_id"]].drop_duplicates().groupby("country_name").size()
nr_ath = pd.DataFrame(nr_ath.reset_index()).rename(columns={0:"nr_participants"})
medal_country_w = medal_country_w.merge(nr_ath, how="left")

Finally calculate proportions of medals by number of participants:

In [ ]:
medal_country_s["total_nr_medals"] = medal_country_s[["Bronze", "Gold", "Silver"]].sum(axis=1)
medal_country_s["prop_medals_participants"] = medal_country_s["total_nr_medals"]/medal_country_s["nr_participants"]

In [ ]:
medal_country_w["total_nr_medals"] = medal_country_w[["Bronze", "Gold", "Silver"]].sum(axis=1)
medal_country_w["prop_medals_participants"] = medal_country_w["total_nr_medals"]/medal_country_w["nr_participants"]

#### 7.1 Number of participants by country

In [ ]:
part = sns.barplot(
    data=medal_country_s.sort_values(by="nr_participants", ascending=False).head(10),
    x="country_name",
    y="nr_participants"
)
part.set_title("Number of participants by country in summer")
plt.xticks(rotation=45)
plt.show()

In [ ]:
part = sns.barplot(
    data=medal_country_w.sort_values(by="nr_participants", ascending=False).head(10),
    x="country_name",
    y="nr_participants"
)
part.set_title("Number of participants by country in winter")
plt.xticks(rotation=45)
plt.show()

**INSIGHTS:**

* The USA was a major contributor of athletes to the olympic games (over 5%), both in summer and in winter.  


#### 7.1 Proportion of medals by country


The **top 10 countries** with more **medals by participant** are:

In [ ]:
part = sns.barplot(
    data=medal_country_s.sort_values(by="prop_medals_participants", ascending=False).head(10),
    x="country_name",
    y="prop_medals_participants"
)
part.set_title("Proportion of medals by participants in summer")
plt.xticks(rotation=45)
plt.show()

In [ ]:
part = sns.barplot(
    data=medal_country_w.sort_values(by="prop_medals_participants", ascending=False).head(10),
    x="country_name",
    y="prop_medals_participants"
)
part.set_title("Proportion of medals by participants in winter")
plt.xticks(rotation=45)
plt.show()

**INSIGHTS:** 

* The USA are clearly superior in summer. They won 72 medals every 100 athletes. 

* Norway is clearly superior in winter. They won 73 medals every 100 athletes.


#### 7.3 Relationship between country latitude and winter games

The data shows that the top 10 countries in the 2014 winter games are located mainly in Northern or Western Europe. Therefore I suspect that countries in high latitudes are better at winter games.

In [ ]:
medal_country_w[["country_name", "Gold", "Silver", "Bronze", "No medal", "nr_participants", "prop_medals_participants"]].sort_values(by="prop_medals_participants", ascending=False).head(10)